In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import xgboost as xgb

In [3]:
# Load dataset
data = pd.read_csv('../data/Telco-Customer-Churn.csv')
data.drop(columns=['customerID'], inplace=True)

In [4]:
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
data['PaymentMethod'] = data['PaymentMethod'].str.replace(' ', '_', regex=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [7]:
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].str.replace(' ', '_', regex=True)

In [8]:
data.columns = data.columns.str.replace(' ', '_', regex=True)

In [9]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [10]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

In [11]:
data['TotalCharges'] = data['TotalCharges'].fillna(0)

In [12]:
data['TotalCharges'].dtypes

dtype('float64')

In [13]:
X = data.drop(columns=['Churn'], axis=1).copy()
y = data['Churn'].copy().map({'No': 0, 'Yes': 1})


In [14]:
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No_phone_service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic_check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One_year,No,Mailed_check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed_check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No_phone_service,DSL,Yes,No,Yes,Yes,No,No,One_year,No,Bank_transfer_(automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber_optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic_check,70.70,151.65,Yes


In [15]:
data['PaymentMethod'].value_counts()

PaymentMethod
Electronic_check             2365
Mailed_check                 1612
Bank_transfer_(automatic)    1544
Credit_card_(automatic)      1522
Name: count, dtype: int64

In [16]:
sum(y)/len(y)

0.2653698707936959

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

pos_rate = y_tr.mean()
print(f"Train prevalence (positives): {pos_rate:.3f}")

Train prevalence (positives): 0.265


In [30]:
#preprocessing
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns

pre_base = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=0.01), cat_cols),
        ('num', 'passthrough', num_cols),
    ],
    remainder='drop'
)


In [32]:

pre = clone(pre_base).fit(X_tr, y_tr)
X_tr_t  = pre.transform(X_tr)
X_val_t = pre.transform(X_val)

pos = y_tr.sum()
neg = len(y_tr) - pos
spw = (neg / pos) if pos > 0 else 1.0


clf = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=3000,    
    learning_rate=0.05,
    max_depth=6,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    n_jobs=-1,
    tree_method='hist',
    random_state=42,
    scale_pos_weight=spw,
    eval_metric='aucpr',
    early_stopping_rounds=50,
)


clf.fit(
    X_tr_t, y_tr,
    eval_set=[(X_val_t, y_val)],
    verbose=False
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3000, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [33]:
val_proba = clf.predict_proba(X_val_t)[:, 1]
val_pred  = (val_proba >= 0.5).astype(int)

In [34]:
print("VAL ROC AUC:", roc_auc_score(y_val, val_proba))
print("VAL PR  AUC:", average_precision_score(y_val, val_proba))
print("VAL Acc   :", accuracy_score(y_val, val_pred))
print("VAL Prec  :", precision_score(y_val, val_pred))
print("VAL Recall:", recall_score(y_val, val_pred))
print("VAL F1    :", f1_score(y_val, val_pred))

VAL ROC AUC: 0.8467597305026415
VAL PR  AUC: 0.6627952058522952
VAL Acc   : 0.7586512866015972
VAL Prec  : 0.5306122448979592
VAL Recall: 0.782608695652174
VAL F1    : 0.6324324324324324


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve

def best_threshold_by_f1(y_true, y_proba):
    p, r, th = precision_recall_curve(y_true, y_proba)
    f1 = (2 * p * r) / (p + r + 1e-12)
    idx = np.nanargmax(f1)
    best_th = th[max(idx-1, 0)] if idx == len(th) else th[idx]
    return best_th, f1[idx], p[idx], r[idx]

def threshold_for_precision(y_true, y_proba, target_p=0.65):
    p, r, th = precision_recall_curve(y_true, y_proba)
    ok = np.where(p[:-1] >= target_p)[0]
    if len(ok) == 0:
        return 0.5  
    return th[ok[0]]

th_f1, f1_at, p_at, r_at = best_threshold_by_f1(y_val, val_proba)
print(f"Best F1 threshold={th_f1:.3f} | F1={f1_at:.3f} | P={p_at:.3f} | R={r_at:.3f}")

th_p65 = threshold_for_precision(y_val, val_proba, target_p=0.65)
print(f"Threshold for ≥65% precision: {th_p65:.3f}")

Best F1 threshold=0.619 | F1=0.653 | P=0.651 | R=0.656
Threshold for ≥65% precision: 0.619
